In [1]:
import numpy as np
import pandas as pd
from numba import jit, vectorize, float64
from time import perf_counter as counter

# Removing bottlenecks with Numba, Cython, and TensorFlow

## Topics

1. Numba to speed up simple operations and create vectorising functions.
2. When to and how to use Cython in its simplest form.
3. How to include these functions in your packages.
4. Tensorflow feature engineering.
5. Quick win for TensorFlow speed in prediction.

## 1. Numba

### Introduction

- Library to translate python code into fast machine code.
- Designed specifically for compatibility with numpy.
- Provides just in time compilation.

### Demo

### Pros v Cons

Pros
- Easy to implement in many cases.
- Significant speed boosts.
- Suited to a lot of data processing needs in data science.

Cons
- Limited scope regarding python libaries.
- When certain functions either do not work or are not accelerated it is difficult to work out why not?
- In more complex use cases needing to make all of the functions compatible is a hassle.

### Speed comparisions

With loops

In [4]:
data = pd.DataFrame(np.random.uniform(0, 1, 1_000_000).reshape(-1,1))
data.columns = ["d"]
data.head()

,d
0,0.638929
1,0.649349
2,0.496717
3,0.614220
4,0.260934


In [5]:
test_values = ["1", "10", "100", "1_000", "10_000", "100_000", "1_000_000"]

In [6]:
empty_results = [np.nan for i in range(7)]

In [7]:
results = {
    "Pure Python": empty_results.copy(),
    "Jit Operation": empty_results.copy(),
    "Jit Apply": empty_results.copy(),
    "Jit Loop": empty_results.copy(),
    "Vectorize": empty_results.copy(),
    "Jit Vectorize": empty_results.copy()
}

Pure python test

In [8]:
def operation(x):
    val = 0
    for i in range(1_000):
        for j in range(1_000):
            val += (x * i) - (x * j) 
    return val

def pure_python_test(n):
    new = np.zeros(n).reshape(-1,1)
    for i in range(n):
        new[i,:] = operation(data.iloc[i,:][0])
    return new

test = "Pure Python"
print(test)
for i, val in enumerate(test_values):
    if i >= 2:
        break
    print(f"Testing {val}")
    baseline_begin = counter()
    pure_python_test(int(val))
    baseline_end = counter()
    results[test][i] = baseline_end-baseline_begin

Pure Python
Testing 1
Testing 10


Jit operation test

In [9]:
@jit
def jit_operation(x):
    val = 0
    for i in range(1_000):
        for j in range(1_000):
            val += (x * i) - (x * j) 
    return val

def jit_operation_test(n):
    new = np.zeros(n).reshape(-1,1)
    d = data.head(n)
    for i in range(n):
        x = d.iloc[i,:][0]
        val = jit_operation(x) 
        new[i,:] = val
    return new

test = "Jit Operation"
print(test)
for i, val in enumerate(test_values):
    if i >= 4:
        break
    print(f"Testing {val}")
    baseline_begin = counter()
    jit_operation_test(int(val))
    baseline_end = counter()
    results[test][i] = baseline_end-baseline_begin

Jit Operation
Testing 1
Testing 10
Testing 100
Testing 1_000


Jit loop test

In [10]:
@jit
def jit_operation(x):
    val = 0
    for i in range(1_000):
        for j in range(1_000):
            val += (x * i) - (x * j) 
    return val

@jit
def jit_loop_test(data):
    new = np.zeros(len(data)).reshape(-1,1)
    for i, val in enumerate(data):
        new[i,:] = val
    return new

test = "Jit Loop"
print(test)
for i, val in enumerate(test_values):
    print(f"Testing {val}")
    baseline_begin = counter()
    jit_loop_test(np.array(data.head(int(val))["d"]))
    baseline_end = counter()
    results[test][i] = baseline_end-baseline_begin

Jit Loop
Testing 1
Testing 10
Testing 100
Testing 1_000
Testing 10_000
Testing 100_000
Testing 1_000_000


Jit apply test

In [11]:
@jit
def jit_operation(x):
    val = 0
    for i in range(1_000):
        for j in range(1_000):
            val += (x * i) - (x * j) 
    return val

def jit_apply_test(n):
    t_d = data.head(n)
    return t_d["d"].apply(jit_operation)

test = "Jit Apply"
print(test)
for i, val in enumerate(test_values):
    if i >= 5:
        break
    print(f"Testing {val}")
    baseline_begin = counter()
    jit_apply_test(int(val))
    baseline_end = counter()
    results[test][i] = baseline_end-baseline_begin

Jit Apply
Testing 1
Testing 10
Testing 100
Testing 1_000
Testing 10_000


Vectorize test

In [12]:
@vectorize([float64(float64)])
def vectorize_operation(x):
    val = 0
    for i in range(1_000):
        for j in range(1_000):
            val += (x * i) - (x * j) 
    return val

def vectorize_test(n):
    t_d = data.head(n)
    new = vectorize_operation(t_d["d"])
    return new

test = "Vectorize"
print(test)
for i, val in enumerate(test_values):
    if i >= 5:
        break
    print(f"Testing {val}")
    baseline_begin = counter()
    vectorize_test(int(val))
    baseline_end = counter()
    results[test][i] = baseline_end-baseline_begin

Vectorize
Testing 1
Testing 10
Testing 100
Testing 1_000
Testing 10_000


Jit Operation with Vectorize

In [13]:
@jit
def jit_operation(x):
    val = 0
    for i in range(1_000):
        for j in range(1_000):
            val += (x * i) - (x * j) 
    return val

@vectorize([float64(float64)])
def jit_vectorize_operation(x):
    return jit_operation(x)

def jit_vectorize_test(n):
    t_d = data.head(n)
    return jit_vectorize_operation(t_d["d"])

test = "Jit Vectorize"
print(test)
for i, val in enumerate(test_values):
    if i >= 5:
        break
    print(f"Testing {val}")
    baseline_begin = counter()
    jit_vectorize_test(int(val))
    baseline_end = counter()
    results[test][i] = baseline_end-baseline_begin

Jit Vectorize
Testing 1
Testing 10
Testing 100
Testing 1_000
Testing 10_000


In [14]:
results_df = pd.DataFrame.from_dict(results, orient="index")
results_df.columns = test_values
results_df.style.set_caption("Speed of function in s (given number of operations)")

,1,10,100,1_000,10_000,100_000,1_000_000
Pure Python,0.490379,4.222536,nan,nan,nan,nan,nan
Jit Operation,0.208173,0.015439,0.128556,1.074879,nan,nan,nan
Jit Apply,0.085609,0.012046,0.110998,1.156075,10.003750,nan,nan
Jit Loop,0.372415,0.000184,0.000087,0.000083,0.000138,0.000656,0.005486
Vectorize,0.002910,0.012810,0.107374,0.993053,9.964653,nan,nan
Jit Vectorize,0.001397,0.011649,0.100765,0.984344,10.869915,nan,nan


## 2. Cython

### Introduction

- The Cython language is a superset of the Python language that additionally supports calling C functions and declaring C types on variables and class attributes. This allows the compiler to generate very efficient C code from Cython code. 
- Write Python code that calls back and forth from and to C or C++ code natively at any point.
- Easily tune readable Python code into plain C performance by adding static type declarations, also in Python syntax.

### Demo 

### Pros v Cons

Pros
- Very fast.
- Extensively supported.
- Utilise C libaries.

Cons
- Need to learn how to write.
- Difficult to optimise.
- Difficulty also rises quickly with complexity.

## 3. Packaging

### Boilerplate

## 4. TensorFlow 

### Demo

# 5. Final questions